In [ ]:
import cv2
import numpy as np

In [39]:
# Define the lower and upper bounds for the color of the shapes you want to detect
# Replace with the lower bounds for the color
lower_color = np.array([0, 0, 0])
# Replace with the upper bounds for the color
upper_color = np.array([255, 255, 255])

# Create an empty dictionary to store the detected shapes
shapes = {}

frame = cv2.imread('trencadiss-petit.jpeg')

# Convert to grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur
blurred = cv2.GaussianBlur(gray, (11, 11), 0) # (7,7)

# Define a larger kernel for morphological operations
kernel = np.ones((10,10), np.uint8) # (5,5)

# Apply dilation
dilated = cv2.dilate(blurred, kernel, iterations = 3) # 1

# Apply erosion
eroded = cv2.erode(dilated, kernel, iterations = 3) # 1

# Apply morphological opening or closing (choose one depending on your needs)
morph = cv2.morphologyEx(eroded, cv2.MORPH_OPEN, kernel)
# morph = cv2.morphologyEx(eroded, cv2.MORPH_CLOSE, kernel)

# Apply Otsu's thresholding
_, thresholded = cv2.threshold(morph, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Apply Canny edge detection on the thresholded image
edges = cv2.Canny(thresholded, 50, 150)

# Find the contours of the shapes
contours, hierarchy = cv2.findContours(
    edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through each contour and store the data
for i, contour in enumerate(contours):
# This is the key line: it approximates the contour
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # Use approx instead of contour for further calculations and drawing
    area = cv2.contourArea(approx)
    perimeter = cv2.arcLength(approx, True)
    x, y, w, h = cv2.boundingRect(approx)
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.drawContours(frame, [approx], 0, (0, 0, 255), 2)
    # area = cv2.contourArea(contour)  # Get the area of the contour
    # # Get the perimeter of the contour
    # perimeter = cv2.arcLength(contour, True)
    # # Get the bounding box of the contour
    # x, y, w, h = cv2.boundingRect(contour)

    # # Draw the bounding box and contour on the original frame
    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # cv2.drawContours(frame, [contour], 0, (0, 0, 255), 2)

    # Store the data in the dictionary
    shape_data = {"area": area, "perimeter": perimeter,
                    "x": x, "y": y, "width": w, "height": h, "contour_points": approx}
    shapes[f"shape{i+1}"] = shape_data

In [40]:
# cv2.imshow('Shapes', thresholded)
cv2.imshow('Shapes', frame)

# Wait for any key to close the window
cv2.waitKey(0)
cv2.destroyAllWindows()


In [41]:
# compute the average color of a shape

def compute_average_colour(shape_data, original_frame):
    x, y, w, h = shape_data['x'], shape_data['y'], shape_data['width'], shape_data['height']
    average_colour = [0,0,0]
    total_pixels = 0
    for i in range(y, y+h):
            for j in range(x, x+w):
                    # check if pixel is i contour
                    if cv2.pointPolygonTest(shape_data["contour_points"], (j, i), False) in [0,1]:
                            average_colour += original_frame[i,j]
                            total_pixels += 1
    average_colour = average_colour / total_pixels
    return average_colour

for shape_data in shapes.values():
    shape_data["average_colour"] = compute_average_colour(shape_data, frame)


In [42]:
# Load the original frame again
original_frame = cv2.imread('trencadiss-petit.jpeg')

# Get the data for the first shape
shape_data = shapes['shape7']

# Get the coordinates and size of the bounding box
x, y, w, h = shape_data['x'], shape_data['y'], shape_data['width'], shape_data['height']
# Draw the bounding box on the original frame
cv2.rectangle(original_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
# Draw the shape on the original frame
cv2.polylines(original_frame, [shape_data["contour_points"]], True, (0, 0, 255), 2)


# paint all contours with their average colour
for shape_data in shapes.values():
    x, y, w, h = shape_data['x'], shape_data['y'], shape_data['width'], shape_data['height']
    cv2.fillPoly(original_frame, [shape_data["contour_points"]], shape_data["average_colour"])
                


# Show the frame
cv2.imshow('Shape 1', original_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# normalize polygons at 0
for shape in shapes.values():
    contour_points = shape["contour_points"]
    # Compute the centroid of the polygon
    centroid = np.mean(contour_points, axis=0)
    # Center the polygon to (0, 0) by subtracting the centroid coordinates
    centered_points = contour_points - centroid
    shape["normalized_polygon"] = [(c[0][0], c[0][1]) for c in centered_points]


In [ ]:
import pygame
import math

# Initialize Pygame
pygame.init()

# Define the size of the window and shapes
window_size = (800, 600)
shape_size = 50

# Calculate the number of shapes and size of the grid
num_shapes = len(shapes)
grid_size = math.ceil(math.sqrt(num_shapes))

# Calculate the total size of the grid
total_grid_size = grid_size * shape_size

# Calculate the top left corner of the grid
grid_x = (window_size[0] - total_grid_size) / 2
grid_y = (window_size[1] - total_grid_size) / 2

screen = pygame.display.set_mode(window_size)
background_color = (0, 0, 0)
screen.fill(background_color)
shape_color = (255, 255, 255)

# Iterate over the shapes
for i, shape in enumerate(shapes.values()):
    # Get the normalized polygon for this shape
    normalized_polygon = shape["normalized_polygon"]

    # Get the width and height of the shape
    w, h = shape["width"], shape["height"]

    # Calculate the scaling factor
    scaling_factor = shape_size / 200 #max(w, h)

    # Calculate the position of this shape in the grid
    shape_x = (i % grid_size) * shape_size + grid_x
    shape_y = (i // grid_size) * shape_size + grid_y

    # Scale and translate the polygon
    positioned_polygon = [(x * scaling_factor + shape_x, 
                           y * scaling_factor + shape_y)
                          for x, y in normalized_polygon]

    # Draw the polygon on the screen
    pygame.draw.polygon(screen, shape_color, positioned_polygon, 1)

# Update the display
pygame.display.flip()

# Wait for the user to close the window
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()